In [1]:
import pandas as pd
import shutil
import os
import numpy as np

In [2]:
SOURCE_DATA_PATH = '../datasets/raw/derm7pt/'
DESTINATION_DATASET_PATH = '../datasets/processed/derm7pt_clinic/'

In [3]:
if not os.path.exists(DESTINATION_DATASET_PATH):

    # if the demo_folder directory is not present
    # then create it
    os.makedirs(DESTINATION_DATASET_PATH, exist_ok=True)

if not os.path.exists(DESTINATION_DATASET_PATH + 'melanoma'):

    # if the demo_folder directory is not present
    # then create it
    os.makedirs(DESTINATION_DATASET_PATH + 'melanoma', exist_ok=True)

if not os.path.exists(DESTINATION_DATASET_PATH + 'benign'):

    # if the demo_folder directory is not present
    # then create it
    os.makedirs(DESTINATION_DATASET_PATH + 'benign', exist_ok=True)

EDA

In [4]:
df = pd.read_csv(SOURCE_DATA_PATH + 'meta/metadata.csv')
df.head()

,case_num,diagnosis,seven_point_score,pigment_network,streaks,pigmentation,regression_structures,dots_and_globules,blue_whitish_veil,vascular_structures,level_of_diagnostic_difficulty,elevation,location,sex,management,clinic,derm,case_id,notes
0,1,basal cell carcinoma,0,absent,absent,absent,absent,absent,absent,arborizing,medium,nodular,abdomen,female,excision,NEL/NEL025.JPG,NEL/Nel026.jpg,NaN,NaN
1,2,basal cell carcinoma,1,absent,absent,absent,absent,irregular,absent,absent,low,palpable,head neck,female,excision,NEL/NEL027.JPG,NEL/Nel028.jpg,NaN,NaN
2,3,basal cell carcinoma,1,absent,absent,absent,absent,irregular,absent,arborizing,medium,palpable,head neck,female,excision,NEL/Nel032.jpg,NEL/Nel033.jpg,NaN,NaN
3,4,basal cell carcinoma,4,absent,absent,absent,blue areas,irregular,present,within regression,low,palpable,lower limbs,male,excision,NEL/NEL034.JPG,NEL/Nel035.jpg,NaN,NaN
4,5,basal cell carcinoma,1,absent,absent,diffuse irregular,absent,absent,absent,absent,high,palpable,upper limbs,female,excision,NEL/NEL036.JPG,NEL/Nel037.jpg,NaN,NaN


In [5]:
df.columns

Index(['case_num', 'diagnosis', 'seven_point_score', 'pigment_network',
       'streaks', 'pigmentation', 'regression_structures', 'dots_and_globules',
       'blue_whitish_veil', 'vascular_structures',
       'level_of_diagnostic_difficulty', 'elevation', 'location', 'sex',
       'management', 'clinic', 'derm', 'case_id', 'notes'],
      dtype='object')

In [6]:
df.location.unique()

array(['abdomen', 'head neck', 'lower limbs', 'upper limbs', 'back',
       'chest', 'acral', 'buttocks', 'genital areas'], dtype=object)

In [7]:
df.loc[df.location == 'acral'].diagnosis.value_counts()

clark nevus                  48
vascular lesion               7
blue nevus                    3
melanoma (in situ)            1
melanoma (0.76 to 1.5 mm)     1
melanoma metastasis           1
reed or spitz nevus           1
Name: diagnosis, dtype: int64

In [8]:
# filtering data in acral regions
acral = df[df.location == 'acral']

In [25]:
acral.diagnosis.value_counts()

clark nevus                  48
vascular lesion               7
blue nevus                    3
melanoma (in situ)            1
melanoma (0.76 to 1.5 mm)     1
melanoma metastasis           1
reed or spitz nevus           1
Name: diagnosis, dtype: int64

In [26]:
acral.head()

,case_num,diagnosis,seven_point_score,pigment_network,streaks,pigmentation,regression_structures,dots_and_globules,blue_whitish_veil,vascular_structures,...,location,sex,management,clinic,derm,case_id,notes,is_dermoscopic,label,lesion_filename
42,43,blue nevus,0,absent,absent,diffuse regular,absent,absent,absent,absent,...,acral,female,no further examination,NFL/Nfl039.jpg,NFL/Nfl040.jpg,NaN,NaN,False,0,datasets/processed/derm7pt_clinic/benign/Nfl03...
48,49,blue nevus,0,absent,absent,diffuse regular,absent,regular,absent,absent,...,acral,male,no further examination,NML/Nml042.jpg,NML/Nml043.jpg,NaN,NaN,False,0,datasets/processed/derm7pt_clinic/benign/Nml04...
51,52,blue nevus,0,absent,absent,diffuse regular,absent,absent,absent,absent,...,acral,male,no further examination,A1l/Aal067.jpg,A1l/Aal068.jpg,NaN,NaN,False,0,datasets/processed/derm7pt_clinic/benign/Aal06...
74,75,clark nevus,0,absent,regular,absent,absent,absent,absent,absent,...,acral,female,clinical follow up,NBL/Nbl096.jpg,NBL/Nbl095.jpg,NaN,NaN,False,0,datasets/processed/derm7pt_clinic/benign/Nbl09...
99,100,clark nevus,2,absent,regular,absent,blue areas,irregular,absent,absent,...,acral,male,excision,NDL/Ndl011.jpg,NDL/Ndl012.jpg,NaN,NaN,False,0,datasets/processed/derm7pt_clinic/benign/Ndl01...


In [30]:
mel = ['melanoma (0.76 to 1.5 mm)',
 'melanoma metastasis',
 'melanoma (in situ)'
]
mel_imgs = acral[acral.diagnosis.isin(mel)].clinic.values
benign_imgs = acral[~acral.diagnosis.isin(mel)].clinic.values

In [31]:
acral['is_dermoscopic'] = False
acral['label'] = acral.diagnosis.apply(lambda x: 1 if x in mel else 0)
acral['lesion_filename'] = acral.clinic.apply(lambda x: DESTINATION_DATASET_PATH[3:] + '{}/'.format('melanoma' if x in mel_imgs else 'benign') + x.split('/')[1])

C:\Users\user\AppData\Local\Temp\ipykernel_7564\2759137817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acral['is_dermoscopic'] = False
C:\Users\user\AppData\Local\Temp\ipykernel_7564\2759137817.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acral['label'] = acral.diagnosis.apply(lambda x: 1 if x in mel else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_7564\2759137817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [28]:
acral[acral.label == 1]

,case_num,diagnosis,seven_point_score,pigment_network,streaks,pigmentation,regression_structures,dots_and_globules,blue_whitish_veil,vascular_structures,...,location,sex,management,clinic,derm,case_id,notes,is_dermoscopic,label,lesion_filename
629,630,melanoma (in situ),1,absent,regular,absent,absent,irregular,absent,absent,...,acral,male,excision,Ggl/Ggl035.jpg,Ggl/Ggl036.jpg,NaN,NaN,False,1,datasets/processed/derm7pt_clinic/melanoma/Ggl...
742,743,melanoma (0.76 to 1.5 mm),7,absent,irregular,diffuse irregular,absent,irregular,present,linear irregular,...,acral,male,excision,NAL/Nal003.jpg,NAL/Nal002.jpg,1a321,NaN,False,1,datasets/processed/derm7pt_clinic/melanoma/Nal...
825,826,melanoma metastasis,1,absent,absent,absent,absent,irregular,absent,absent,...,acral,male,excision,FCL/Fcl015.jpg,FCL/Fcl016.jpg,NaN,NaN,False,1,datasets/processed/derm7pt_clinic/melanoma/Fcl...


In [29]:
# save metadata
acral.to_csv(DESTINATION_DATASET_PATH + 'metadata.csv', index=False)

Moving filtered data

In [32]:
for img_path in benign_imgs:
    shutil.copy(SOURCE_DATA_PATH + 'images/'+img_path, DESTINATION_DATASET_PATH + 'benign/')

for img_path in mel_imgs:
    shutil.copy(SOURCE_DATA_PATH + 'images/'+img_path, DESTINATION_DATASET_PATH + 'melanoma/')